In [9]:
import tensorflow as tf
import cv2
from cvzone.HandTrackingModule import HandDetector
import matplotlib.pyplot as plt
import numpy as np
import math
import tensorflow.keras
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [10]:
class Classifier:

    def __init__(self, modelPath, labelsPath=None):
        self.model_path = modelPath
        np.set_printoptions(suppress=True)
        # Load the model
        self.model = tensorflow.keras.models.load_model(self.model_path)
        
        self.data = np.ndarray(shape=(1, 64, 64, 1), dtype=np.float32)
        self.labels_path = labelsPath
        if self.labels_path:
            label_file = open(self.labels_path, "r")
            self.list_labels = []
            for line in label_file:
                stripped_line = line.strip()
                self.list_labels.append(stripped_line)
            label_file.close()
        else:
            print("No Labels Found")

    def getPrediction(self, img, draw= True, pos=(50, 50), scale=2, color = (0,255,0)):
        imgS = cv2.resize(img, (64, 64))
        gray = cv2. cvtColor(imgS, cv2.COLOR_RGB2GRAY)
        gray = np.expand_dims(gray, 2)

        # turn the image into a numpy array
        image_array = np.asarray(gray)
        # Normalize the image
        normalized_image_array = (image_array.astype(np.float32) / 255)

        # Load the image into the array
        self.data[0] = normalized_image_array

        # run the inference
        prediction = self.model.predict(self.data)
        indexVal = np.argmax(prediction, axis=1)
        print(prediction, indexVal)

        if draw and self.labels_path:
            cv2.putText(img, str(self.list_labels[indexVal]),
                        pos, cv2.FONT_HERSHEY_COMPLEX, scale, color, 2)

        return list(prediction[0]), indexVal[0]

In [12]:
cap = cv2.VideoCapture(1)
detector = HandDetector(maxHands=1)

classifier = Classifier("model/imageclassifier9lettersV2_2.h5", "model/labels4.txt")

offset = 20
imgSize = 300

folder = "letras"
counter = 0

labels = ["A", "B", "C", "D", "E", "F", "G", "I", "L"]

while True:
    success, img = cap.read()
    imgOutput = img.copy()
    hands, img = detector.findHands(img)
    if hands:
        hand = hands[0]
        x, y, w, h = hand['bbox']

        imgWhite = np.ones((imgSize, imgSize, 3), np.uint8) * 255
        imgCrop = img[y - offset:y + h + offset, x - offset:x + w + offset]

        imgCropShape = imgCrop.shape

        aspectRatio = h / w

        if aspectRatio > 1:
            k = imgSize / h
            wCal = math.ceil(k * w)
            imgResize = cv2.resize(imgCrop, (wCal, imgSize))
            imgResizeShape = imgResize.shape
            wGap = math.ceil((imgSize - wCal) / 2)
            imgWhite[:, wGap:wCal + wGap] = imgResize
            prediction, index = classifier.getPrediction(imgWhite, draw=False)
            cv2.rectangle(imgOutput, (x - offset, y - offset-50),
                      (x - offset+90, y - offset-50+50), (255, 0, 255), cv2.FILLED)
            cv2.putText(imgOutput, labels[index], (x, y -26), cv2.FONT_HERSHEY_COMPLEX, 1.7, (255, 255, 255), 2)
            cv2.rectangle(imgOutput, (x-offset, y-offset),
                      (x + w+offset, y + h+offset), (255, 0, 255), 4)

        else:
            k = imgSize / w
            hCal = math.ceil(k * h)
            imgResize = cv2.resize(imgCrop, (imgSize, hCal))
            imgResizeShape = imgResize.shape
            hGap = math.ceil((imgSize - hCal) / 2)
            imgWhite[hGap:hCal + hGap, :] = imgResize

        cv2.imshow("ImageCrop", imgCrop)
        cv2.imshow("ImageWhite", imgWhite)

    cv2.imshow("Image", imgOutput)
    cv2.waitKey(1)

1/1 [==============================] - 0s 82ms/step
[[0.00418412 0.19436808 0.01719863 0.0253622  0.49730033 0.16842994
  0.02188586 0.0693344  0.00193658]] [4]
1/1 [==============================] - 0s 20ms/step
[[0.00689766 0.20305152 0.02601835 0.02262026 0.38599774 0.2505227
  0.02433276 0.07710353 0.00345546]] [4]
1/1 [==============================] - 0s 22ms/step
[[0.00283283 0.2039713  0.00738472 0.03599018 0.5956693  0.07909379
  0.02367817 0.05058683 0.00079281]] [4]
1/1 [==============================] - 0s 21ms/step
[[0.00042869 0.5268464  0.00458195 0.01629429 0.2042143  0.22112525
  0.00475338 0.02161326 0.00014238]] [1]
1/1 [==============================] - 0s 25ms/step
[[0.00026665 0.47610232 0.00815999 0.00643011 0.1210197  0.37544137
  0.00238512 0.01003702 0.00015767]] [1]
1/1 [==============================] - 0s 26ms/step
[[0.00026616 0.47253636 0.01086353 0.00638356 0.14170079 0.35681367
  0.00226366 0.00901013 0.00016216]] [1]
1/1 [==============================

1/1 [==============================] - 0s 19ms/step
[[0.         0.00000002 0.9999995  0.         0.00000032 0.
  0.         0.         0.00000012]] [2]
1/1 [==============================] - 0s 19ms/step
[[0.         0.00000003 0.9999995  0.         0.00000028 0.
  0.         0.         0.00000013]] [2]
1/1 [==============================] - 0s 20ms/step
[[0.         0.00000006 0.9999995  0.         0.00000028 0.
  0.         0.         0.00000013]] [2]
1/1 [==============================] - 0s 20ms/step
[[0.         0.00000008 0.99999917 0.         0.0000005  0.00000001
  0.         0.         0.00000029]] [2]
1/1 [==============================] - 0s 20ms/step
[[0.         0.0000001  0.9999993  0.         0.00000042 0.00000001
  0.         0.         0.00000018]] [2]
1/1 [==============================] - 0s 19ms/step
[[0.         0.00000006 0.9999994  0.         0.0000004  0.00000001
  0.         0.         0.00000015]] [2]
1/1 [==============================] - 0s 19ms/step
[[0.  

1/1 [==============================] - 0s 18ms/step
[[0.08525691 0.00008844 0.00019164 0.00024652 0.9126965  0.00019074
  0.00124108 0.00000762 0.00008056]] [4]
1/1 [==============================] - 0s 18ms/step
[[0.1051836  0.00010238 0.00021828 0.00035585 0.8922548  0.00019425
  0.00157166 0.00000973 0.00010947]] [4]
1/1 [==============================] - 0s 18ms/step
[[0.05738204 0.0001509  0.00008198 0.0002972  0.9415901  0.00012269
  0.00035408 0.00000722 0.00001386]] [4]
1/1 [==============================] - 0s 18ms/step
[[0.05939007 0.00009178 0.0001368  0.00020374 0.939929   0.00006623
  0.00016476 0.0000045  0.00001312]] [4]
1/1 [==============================] - 0s 18ms/step
[[0.05376433 0.00025323 0.00015152 0.00041359 0.94513935 0.00007789
  0.00017992 0.00000888 0.00001125]] [4]
1/1 [==============================] - 0s 18ms/step
[[0.03159816 0.0001524  0.00013741 0.00018335 0.9678091  0.00003832
  0.00007277 0.00000351 0.00000507]] [4]
1/1 [=============================

1/1 [==============================] - 0s 24ms/step
[[0.00030935 0.00721704 0.00000488 0.11556295 0.00439852 0.00134095
  0.01761015 0.85352695 0.00002924]] [7]
1/1 [==============================] - 0s 29ms/step
[[0.00041167 0.00912304 0.0000079  0.15153241 0.00622983 0.00154872
  0.02838138 0.8027109  0.00005419]] [7]
1/1 [==============================] - 0s 49ms/step
[[0.00006578 0.01879654 0.00000176 0.47095388 0.0012333  0.00039541
  0.01456014 0.4939769  0.00001622]] [7]
1/1 [==============================] - 0s 36ms/step
[[0.00017362 0.00542274 0.0000011  0.7701781  0.0018159  0.00004469
  0.01808242 0.2042747  0.00000674]] [3]
1/1 [==============================] - 0s 30ms/step
[[0.0000486  0.02733916 0.00000079 0.6217682  0.00097387 0.00027151
  0.00594726 0.3436454  0.00000526]] [3]
1/1 [==============================] - 0s 34ms/step
[[0.00010986 0.02128386 0.00000128 0.4240452  0.00146151 0.00049775
  0.00973885 0.5428521  0.00000958]] [7]
1/1 [=============================

1/1 [==============================] - 0s 18ms/step
[[0.00001299 0.01128336 0.00000156 0.449727   0.0012464  0.00084608
  0.00945017 0.5274266  0.00000585]] [7]
1/1 [==============================] - 0s 18ms/step
[[0.00002191 0.0086235  0.00000206 0.33572772 0.00167782 0.00102621
  0.01118308 0.64173096 0.00000679]] [7]
1/1 [==============================] - 0s 19ms/step
[[0.00002124 0.00717098 0.00000164 0.25073445 0.00157082 0.00125754
  0.008957   0.7302807  0.00000569]] [7]
1/1 [==============================] - 0s 18ms/step
[[0.00000946 0.01421209 0.00000096 0.41861957 0.00099276 0.00098083
  0.00706663 0.5581147  0.00000301]] [7]
1/1 [==============================] - 0s 18ms/step
[[0.00000999 0.01361475 0.00000078 0.2731425  0.00101335 0.00140573
  0.00433508 0.70647585 0.000002  ]] [7]
1/1 [==============================] - 0s 18ms/step
[[0.00000845 0.01332697 0.00000089 0.35359403 0.00091894 0.00112764
  0.00561045 0.6254101  0.00000251]] [7]
1/1 [=============================

...

1/1 [==============================] - 0s 33ms/step
[[0.01940054 0.00393089 0.29629645 0.00036835 0.6244941  0.00481744
  0.01989353 0.007632   0.02316671]] [4]
1/1 [==============================] - 0s 34ms/step
[[0.01677766 0.00302856 0.38881522 0.00028627 0.5318197  0.00368705
  0.01924274 0.00753741 0.02880531]] [4]
1/1 [==============================] - 0s 27ms/step
[[0.01894488 0.00309809 0.32454365 0.00029643 0.603822   0.00390316
  0.01710622 0.00604327 0.02224236]] [4]
1/1 [==============================] - 0s 28ms/step
[[0.01165597 0.00210915 0.4737008  0.00016979 0.46898448 0.00301614
  0.01187383 0.00447755 0.02401231]] [2]
1/1 [==============================] - 0s 28ms/step
[[0.01347282 0.00256556 0.38992968 0.00021041 0.55034935 0.00356215
  0.01281885 0.00538083 0.02171033]] [4]
1/1 [==============================] - 0s 33ms/step
[[0.01783239 0.00293581 0.3204425  0.00028091 0.6101549  0.0042238
  0.015606   0.00658221 0.02194151]] [4]
1/1 [==============================

1/1 [==============================] - 0s 33ms/step
[[0.00354938 0.00103442 0.00013264 0.00006294 0.4251693  0.5643827
  0.00044231 0.00509539 0.00013094]] [5]
1/1 [==============================] - 0s 29ms/step
[[0.00280703 0.00078104 0.00009836 0.0000432  0.36554095 0.62649
  0.00028166 0.00385717 0.00010063]] [5]
1/1 [==============================] - 0s 30ms/step
[[0.00351934 0.00074659 0.00009597 0.00004726 0.42193016 0.5693915
  0.00030561 0.00386381 0.00009975]] [5]
1/1 [==============================] - 0s 28ms/step
[[0.00395576 0.000898   0.00013854 0.00007298 0.45780718 0.5305442
  0.00042866 0.00600403 0.00015054]] [5]
1/1 [==============================] - 0s 35ms/step
[[0.00434127 0.00066916 0.00013258 0.00005738 0.47080842 0.518331
  0.00037984 0.00513617 0.00014422]] [5]
1/1 [==============================] - 0s 33ms/step
[[0.00389218 0.00087836 0.00017305 0.00007696 0.4968779  0.49138755
  0.00045701 0.00609301 0.00016399]] [4]
1/1 [==============================] - 0s 

1/1 [==============================] - 0s 33ms/step
[[0.01727024 0.00061057 0.00023875 0.42856383 0.0946952  0.00378439
  0.10400438 0.3505739  0.00025884]] [3]
1/1 [==============================] - 0s 28ms/step
[[0.02577487 0.00037835 0.00067899 0.2125316  0.28252205 0.00777776
  0.11342473 0.35658327 0.00032845]] [7]
1/1 [==============================] - 0s 28ms/step
[[0.00952996 0.00062917 0.00064986 0.15450898 0.28574067 0.01806877
  0.05522387 0.47549552 0.00015324]] [7]
1/1 [==============================] - 0s 28ms/step
[[0.0090515  0.00061227 0.00030705 0.13748111 0.15502004 0.0195192
  0.0496046  0.62827575 0.00012843]] [7]
1/1 [==============================] - 0s 28ms/step
[[0.01005512 0.00072891 0.00025212 0.20675741 0.20549372 0.00981466
  0.04791249 0.5188857  0.00009987]] [7]
1/1 [==============================] - 0s 32ms/step
[[0.00754962 0.00082971 0.00032065 0.1865666  0.16540073 0.01644973
  0.0519036  0.5708637  0.00011559]] [7]
1/1 [==============================

1/1 [==============================] - 0s 33ms/step
[[0.00226671 0.00001803 0.00036689 0.00052134 0.00036492 0.00000344
  0.35789698 0.00022635 0.6383353 ]] [8]
1/1 [==============================] - 0s 28ms/step
[[0.00135536 0.0000074  0.00014439 0.00027658 0.00015142 0.00000114
  0.39339784 0.00015379 0.6045121 ]] [8]
1/1 [==============================] - 0s 25ms/step
[[0.00255939 0.00001189 0.00012651 0.00081336 0.00017739 0.00000179
  0.42669153 0.00053968 0.56907856]] [8]
1/1 [==============================] - 0s 28ms/step
[[0.00101069 0.00017573 0.00263023 0.00061862 0.0004819  0.00003433
  0.01425945 0.00136266 0.97942644]] [8]
1/1 [==============================] - 0s 60ms/step
[[0.979715   0.00000244 0.00016921 0.00014914 0.00365019 0.00000517
  0.00227436 0.00006982 0.01396468]] [0]
1/1 [==============================] - 0s 30ms/step
[[0.99738926 0.00000004 0.00000085 0.00001561 0.00201493 0.00000058
  0.00046388 0.00002126 0.00009355]] [0]
1/1 [=============================

KeyboardInterrupt: 